In [1]:
using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    ŷ = model(x)
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

loss_and_accuracy (generic function with 1 method)

In [2]:
# Opracowane na podstawie https://github.com/FluxML/model-zoo/blob/3e91af32ebfad628b616618b11bfff2f9f519bec/vision/conv_mnist/conv_mnist.jl
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :) # insert trivial channel dim
    yhot  = Flux.onehotbatch(data.targets, 0:9)  # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    Conv((3, 3), 1 => 6,  relu),
    MaxPool((2, 2)),
    Conv((3, 3), 6 => 16, relu),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(400 => 84, relu), 
    Dense(84 => 10, identity),
)

#= ew. prostsza architektura:

net = Chain(
    Conv((3, 3), 1 => 6,  relu, bias=false),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(13*13*6 => 84, relu, bias=false), 
    Dense(84 => 10, identity, bias=false)
)
=#

Chain(
  Conv((3, 3), 1 => 6, relu),           # 60 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 6 => 16, relu),          # 880 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(400 => 84, relu),               # 33_684 parameters
  Dense(84 => 10),                      # 850 parameters
)                   # Total: 8 arrays, 35_474 parameters, 139.773 KiB.

In [12]:
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1)


400×1 Matrix{Float32}:
 0.0
 0.07217326
 0.0
 0.0
 0.0
 0.045555223
 0.026632763
 0.0
 0.0
 0.0
 ⋮
 0.51805997
 0.26482785
 0.545838
 0.5025322
 0.2411429
 0.5090034
 0.32289258
 0.40637982
 0.20456274

In [ ]:
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 1e-2,
    epochs = 3,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

In [ ]:
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 1e-2,
    epochs = 3,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

In [9]:
for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        grads = Flux.gradient(model -> Flux.logitcrossentropy(model(x), y), net)
        sizeof(grads)
        #Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

  8.676747 seconds (440.60 k allocations: 8.012 GiB, 5.27% gc time, 3.72% compilation time)


┌ Info: 1
│   acc = 10.62
│   test_acc = 10.72
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:10


  8.037630 seconds (303.35 k allocations: 8.004 GiB, 5.22% gc time)


┌ Info: 2
│   acc = 10.62
│   test_acc = 10.72
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:10


  7.939930 seconds (303.35 k allocations: 8.004 GiB, 5.50% gc time)


┌ Info: 3
│   acc = 10.62
│   test_acc = 10.72
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:10


In [6]:
train_log

3-element Vector{Any}:
 (epoch = 1, loss = 2.3029556f0, acc = 10.62, test_loss = 2.3041186f0, test_acc = 10.72)
 (epoch = 2, loss = 2.3029556f0, acc = 10.62, test_loss = 2.3041186f0, test_acc = 10.72)
 (epoch = 3, loss = 2.3029556f0, acc = 10.62, test_loss = 2.3041186f0, test_acc = 10.72)